In [62]:
import pickle
import pandas as pd

#function to make prediction given home team and away team
model_file = './models/naive_bayes.sav'

# load the model from disk
model = pickle.load(open(model_file, 'rb'))
current_team_df =  pd.read_csv('./data/current_team_data.csv')
current_team_df = current_team_df.drop(current_team_df.columns[[0]],axis = 1)

def predict(home_team, away_team):
    home = current_team_df.loc[current_team_df['Team'] == home_team].iloc[0,1:]
    home.index = list(map(lambda n: "home_" + n, home.index.to_list()))

    away = current_team_df.loc[current_team_df['Team'] == away_team].iloc[0,1:]

    away.index = list(map(lambda n: "away_" + n, away.index.to_list()))
    X = pd.concat([home, away])
    X = X.to_frame().T

    #put data in form for model
    y_pred = model.predict(X)[0]
    y_pred_prob = model.predict_proba(X)[0][1 if y_pred else 0]
    #print("We predict the home team " + home_team + str(" WINS" if y_pred else " LOSES") + " with an probability of " + str(round(y_pred_prob, 2)))
    return model.predict_proba(X)[0][1]

#go through current schedule
#for any game that doesn't have a score predict the winner
#save the output in a json file for website
schedule_df = pd.read_csv("./data/schedule.csv")
schedule_df = schedule_df.drop(schedule_df.columns[[0]],axis = 1)

for i in range(schedule_df.shape[0]):
    #row has no prediction
    if pd.isna(schedule_df.iloc[i].get('Home_Pts')):
        prediction = predict(schedule_df.iloc[i,2], schedule_df.iloc[i,4])
        schedule_df.iloc[i, 6]  = round(prediction, 2)
schedule_df.to_csv("./data/schedule.csv")
        

In [69]:
#convert schedule.csv to a readble javascript object for the website
import pandas as pd
from datetime import datetime
schedule_df = pd.read_csv("./data/schedule.csv")
schedule_df = schedule_df.drop(schedule_df.columns[[0]],axis = 1)

data = {}

for i in range(schedule_df.shape[0]):
    game = {}
    game["time"] = schedule_df.iloc[i, 1] if not pd.isna(schedule_df.iloc[i, 1]) else ""
    game["homeTeam"] = schedule_df.iloc[i, 2] if not pd.isna(schedule_df.iloc[i, 2]) else ""
    game["homeTeamPoints"] = schedule_df.iloc[i, 3] if not pd.isna(schedule_df.iloc[i, 3]) else ""
    game["awayTeam"] = schedule_df.iloc[i, 4] if not pd.isna(schedule_df.iloc[i, 4]) else ""
    game["awayTeamPoints"] = schedule_df.iloc[i, 5] if not pd.isna(schedule_df.iloc[i, 5]) else ""
    game["prediction"] = schedule_df.iloc[i, 6] if not pd.isna(schedule_df.iloc[i, 6]) else ""
    

    #make it a blank list if it doesn't exist
    date = schedule_df.iloc[i, 0]
    #add zero padded days and months
    date = datetime.strptime(date, '%m/%d/%Y').strftime('%m/%d/%Y')
    if data.get(date, None) != None:
        data[date].append(game)
    else:
        data[date] = [game]

with open('./website/data.js', "w") as file:
    file.write("const data = " + str(data))
